### Load libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime

# setup tensorboard for logging
from torch.utils.tensorboard import SummaryWriter

# local imports
import sys
sys.path.append('../')
from datasets import SpectraDataset
from models import MLP, ResidualMLP
from loops import train,val

### Define params

In [ ]:
filename = '../DL-Assisted-NHA-Inverse-Design-/Dataset 6655.csv'
test_val_split = 0.1  # portion of data assigned to validation set
batch_size = 64  # batch size
lr = 1.1e-4  # learning rate
hidden_layers = [5, 2000, 2000, 2000, 2000, 2000, 200]  # structure of the neural network
epochs = 4000

### Prepare datasets

#### Load data into dataframe and clean them

In [ ]:
df = pd.read_csv(filename)
df['Spectra'] = df.values[:,5:][:,::-1].tolist()
df['Spectra'] = df['Spectra'].apply(np.array)
df.drop(df.columns[5:-1], axis=1, inplace=True)
df.columns = ['Lattice','Material','Thickness','Radius','Pitch','Spectra']

#### Split dataframe in train/val and features/labels (X/y) 

In [ ]:
X_df = df[['Lattice','Material','Thickness','Radius','Pitch']]
X_df = X_df/X_df.max()
y_df = df['Spectra']
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=test_val_split, random_state=42)

In [ ]:
X_df

#### Create datasets

In [ ]:
training_data = SpectraDataset(X_train,y_train)
val_data = SpectraDataset(X_val,y_val)

#### Create dataloaders

In [ ]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle= True)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

### Training

#### Choosing training device: cpu, gpu, etc...

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#### Define model

In [ ]:
# model = MLP(hidden_layers=hidden_layers).to(device)
model = ResidualMLP(hidden_layers=hidden_layers).to(device)

#### Defining loss and optimizer

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

#### Run training loop

In [ ]:
# create summary writer for tensorboard
now = datetime.now()  # current date and time
date_time = now.strftime("%d%m%y_%H%M%S")
writer_path = '../tb_logs/' + date_time + '/'
writer = SummaryWriter(writer_path)

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = train(train_dataloader, model, loss_fn, optimizer, device)
    val_loss = val(val_dataloader, model, loss_fn, device)
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
writer.flush()
writer.close()
print("Done!")

In [ ]:
model.eval()

In [ ]:
iter_val = iter(val_dataloader)
val_sample = next(iter_val)

In [ ]:
val_sample

In [ ]:
n_sample = 59
val_feature = val_sample[0][n_sample:n_sample+1]
val_label = val_sample[1][n_sample:n_sample+1]
nn_label = model(val_feature.to(device))

In [ ]:
val_feature

In [ ]:
plt.plot(val_label.T)
plt.plot(nn_label.cpu().detach().T)

In [ ]:
torch.sum((val_label-nn_label.cpu())**2)/200